# Data Preprocessing

In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
import pickle
from random import shuffle
import matplotlib.pyplot as plt
import datetime

In [ ]:
dataset = os.path.join(os.getcwd(), "pdataset")
data = []
for i , value in enumerate(os.listdir(dataset)):
    print(value)
    f_dir = os.path.join(dataset,value)
    for each in os.listdir(f_dir):
        image = cv2.imread(os.path.join(f_dir,each),0)
        image = cv2.resize(image,(200,200))
        data.append((image,i))

In [ ]:
shuffle(data)
x = []
y = []
for features , label in data:
    x.append(features)
    y.append(label)
x = np.array(x).reshape(-1,200,200,1)
pickle_out = open("X.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close()

pickle_out = open("Y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

# Training

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import pickle

In [ ]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

X = X/255.0

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  

model.add(Dense(64))

model.add(Dense(1,activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

logdir = "logs\\scalars\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
model.fit(X, y, batch_size=32, epochs=50, validation_split=0.3,  callbacks=[tensorboard_callback])

model.save("model.h5")

Train on 540 samples, validate on 232 samples
Epoch 1/50
540/540 [==============================] - 2s 4ms/sample - loss: 0.6725 - acc: 0.6333 - val_loss: 0.6596 - val_acc: 0.6336
Epoch 2/50
540/540 [==============================] - 2s 3ms/sample - loss: 0.6642 - acc: 0.6333 - val_loss: 0.6535 - val_acc: 0.6336
Epoch 3/50
540/540 [==============================] - 2s 3ms/sample - loss: 0.6412 - acc: 0.6333 - val_loss: 0.6614 - val_acc: 0.6336
Epoch 4/50
540/540 [==============================] - 2s 3ms/sample - loss: 0.6317 - acc: 0.6463 - val_loss: 0.6386 - val_acc: 0.6336
Epoch 5/50
540/540 [==============================] - 2s 3ms/sample - loss: 0.6024 - acc: 0.6444 - val_loss: 0.6187 - val_acc: 0.6810
Epoch 6/50
540/540 [==============================] - 2s 3ms/sample - loss: 0.5746 - acc: 0.6815 - val_loss: 0.5990 - val_acc: 0.6681
Epoch 7/50
540/540 [==============================] - 2s 3ms/sample - loss: 0.5313 - acc: 0.7222 - val_loss: 0.5923 - val_acc: 0.6724
Epoch 8/50
540/5

# Testing

In [ ]:
model = tf.keras.models.load_model('model.h5')
model.summary()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def ptest(path):
    image = cv2.imread(path,0)
    image = cv2.resize(image,(200,200))
    x = np.array(image).reshape(-1,200,200,1)
    y = model.predict_classes(x)
    return y[0][0]

In [ ]:
for i , each in enumerate(os.listdir("ptest")): #[ "normal" = 0,"stroke" = 1]
    print(each,f"expected {i} : observerd ->")

    for file in os.listdir(f"ptest/{each}"):
        print(file," -> ",ptest(f"ptest/{each}/"+file))
        